# Red convolucional - Inception

1. Introducción Inception.
2. Provision de imágenes a la arquitectura. Pre procesado de imágenes.
3. Transferencia de aprendizaje: descriptores profundos
    * Clasificación
    * Medidas de desempeño
4. Transferencia de aprendizaje: Fine tuning
    * Medidas de desempeño
5. Resumen. Discusión resultados

## LISTADO MEJORAS

* numero fotos por funcion no especificando constante
* mejorar organizacion imports
* guardar en fichero los deepfeatures hdf5
* incorparar diseño de la estructura de los folders e imagenes
* preprocesamiento imagenes con tensorflow directamente

## 1.Introducción Inception.

In [ ]:
# --> To do --> checkpoint file explaining!!!!

## 2.Provision de imágenes a la arquitectura. Pre procesado de imágenes.

In [ ]:
# path structure

In [ ]:
# architecture paths
ARCHITECTURES = "pretrained-model/"
INCEPTION_V3_PATH = "pretrained-model/inception_v3.ckpt"
INCEPTION_V4_PATH = "pretrained-model/inception_v4.ckpt"

# image dataset path
PATH_TO_IMG = "../img-raw"

# img data --> To do --> get automated
WIDTH = 299
HEIGHT = 299
CHANNELS = 3

# CONST --> To do --> get automated
N_IMG = 5000 # number of imgs
N_FEATURES =  2048 # number of deepfeatures


In [ ]:
# imports --> To do --> get comments of each one of them, load them when they are going to be used?
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import os
import scipy
import sys
import re

# general, prepocessing
import numpy as np

# preprocessing
from PIL import Image

# check them out
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import pandas as pd

#some errors
# import sklearn
# from sklearn import cross_validation
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.svm import SVC, LinearSVC
# import skimage.io
# import skimage.transform
# from scipy.misc import imresize
# from skimage.transform import resize #al final puede q no se use

In [ ]:
# img categories --> subfolders
# the images are tagged by the name folder --> To do --> wrap it in a function instead
imgCategories = sorted([category for category in os.listdir(PATH_TO_IMG)
                  if os.path.isdir(os.path.join(PATH_TO_IMG, category))])

#imgCategories

In [ ]:
# img list + cat
from collections import defaultdict

imgPaths = defaultdict(list) #diccionario categoria - path/img.tif en test hay link a esta clase

for imgCategory in imgCategories:
    imgFolder = os.path.join(PATH_TO_IMG, imgCategory)
    for img in os.listdir(imgFolder):
        if img.endswith(".tif"): #just in case there are other kind of files
            # due win backslash management method replace is used
            imgPaths[imgCategory].append(os.path.join(imgFolder, img).replace("\\","/"))

#sorting the img names
for imgPath in imgPaths.values():
    imgPath.sort()  

#list(imgPaths.values()) #listado de fotos usado para el feed de la cnn
#len(list(imgPaths.values())[1]) #numero de imagenes de la categoria 1
#type(list(imgPaths.values()))
#list(imgPaths.items()) # listado del diccionario categoria + fotos de categoria




In [ ]:
# only the img listed, so this way via regex we get the name of the category
imgList = []

for imgCategory in imgCategories:
    imgFolder = os.path.join(PATH_TO_IMG, imgCategory)
    
    for img in os.listdir(imgFolder):
        if img.endswith(".tif"):
            imgList.append(os.path.join(imgFolder, img).replace("\\","/"))

# print(imgList[0]) # first element

# print(re.split('_\d+',imgList[0].split('/')[2])[0]) #cat of first element



In [ ]:
# img pre procesing function, will be called later on

# just in case that tf decoder jpg does not work, we should change the img tif to jpg
# https://stackoverflow.com/questions/34484148/feeding-image-data-in-tensorflow-for-transfer-learning
# import numpy as np
# from PIL import Image
# # ...

# image = Image.open("example.png")
# image_array = np.array(image)[:, :, 0:3]  # Select RGB channels only.

# prediction = sess.run(softmax_tensor, {'DecodeJpeg:0': image_array})


def img_preprocessing(img, width, height):
    
    img = imresize(img, (width, height)) # just resize in this architecture --> enlarge both dims
    img = img.astype(np.float32) / 255 #colors as 32-bit [0.0,1.0] a veces indican q tiene q estar entre -1 y 1
    
    return img



In [ ]:
# img pre procesing function with tensorflow, will be called later on (calc time improvement¿?)
def img_tf_preprocessing(img, width, height):

    imgBatch = tf.expand_dims(img, 0)
    imgBatch = tf.image.resize_bilinear(img_batch, [height, width])
    img = imgBatch[0] / 255  # back to a single image, and scale the colors from 0.0 to 1.0
    
    return img

## 3.Transferencia de aprendizaje: descriptores profundos

# pruebas con keras https://github.com/fchollet/keras


In [ ]:
# .keras/keras.json
# {
#     "floatx": "float32",
#     "epsilon": 1e-07,
#     "backend": "tensorflow",
#     "image_data_format": "channels_last" 
# }

# channels_last input format --> (150,150,3)

# in anaconda environment --> set "KERAS_BACKEND=tensorflow" (problems reading keras.json)

In [2]:
import numpy as np
import scipy
from keras.applications.inception_v3 import InceptionV3
#from keras.preprocessing import image #error de windows...
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [3]:
# example // input --> (299, 299, 3) // pooling none (default)
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150,150,3))


# loading an image + preprocessing 
example = '../img-raw/01_TUMOR/1198_CRC-Prim-HE-07.tif_Row_301_Col_901.tif'
img = image.load_img(example, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) #¿?
x = preprocess_input(x)

features = model.predict(x)

# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input
# import numpy as np

# model = VGG16(weights='imagenet', include_top=False)

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# features = model.predict(x)

KeyboardInterrupt: 

In [ ]:
print(features)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = '../img-raw/01_TUMOR/1198_CRC-Prim-HE-07.tif_Row_301_Col_901.tif'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

# todo lo de abajo en ppio no funciona

In [ ]:
def create_graph():

    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")

    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=False)

    # predictions = end_points["Predictions"]

    saver = tf.train.Saver()

    return saver, logits, end_points



In [ ]:
saver, logits, end_points = create_graph()

In [ ]:
end_points["PreLogits"]

In [ ]:
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

prelogits #en teoria tengo los deep features


In [ ]:
X_test = test_image.reshape(-1, height, width, channels)

with tf.Session() as sess:
    saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    predictions_val = predictions.eval(feed_dict={X: X_test})


    X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()


# extract all features from pool layer of InceptionV3
def extract_features(imgList):
    nb_features = 2048
    features = np.empty((len(imgList),nb_features))
    labels = []
    
    #saver = create_graph()
    create_graph()
    
    with tf.Session() as sess:
        
        saver.restore(sess, INCEPTION_V3_PATH)
        
        next_to_last_tensor = sess.graph.get_tensor_by_name('PreLogits')
        
        
        for ind, image in enumerate(imgList):
            print('Processing %s...' % (image))
#             if not gfile.Exists(image):
#                 tf.logging.fatal('File does not exist %s', image)
                
            #pre processing
            image = img_preprocessing(image, 299, 299)
            
            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor,
            {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',imgList[0].split('/')[2])[0])
        return features, labels #end for

with tf.Session() as sess:
    saver.restore(sess, INCEPTION_V3_PATH)
    
    
    
    

In [ ]:
# https://github.com/taey16/tf/tree/master/imagenet --> classify_image_graph_def.pb
def create_graph():
    with gfile.FastGFile(os.path.join(ARCHITECTURES, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')


In [ ]:
# extract all features from pool layer of InceptionV3
def extract_features(imgList):
    nb_features = 2048
    features = np.empty((len(imgList),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for ind, image in enumerate(imgList):
#             if (ind%100 == 0):
#                 print('Processing %s...' % (image))
    
#             if not gfile.Exists(image):
#                 tf.logging.fatal('File does not exist %s', image)

            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',image.split('/')[1])[0])

    return features, labels

In [ ]:
#function call
features,labels = extract_features(imgList)

#save info to pickle
pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

features = pickle.load(open('features'))
labels = pickle.load(open('labels'))

In [ ]:
# save matrix to hdf5 vs pickle
#http://docs.h5py.org/en/latest/quick.html
# https://stackoverflow.com/questions/29209293/how-to-store-an-array-in-hdf5-file-which-is-too-big-to-load-in-memory

# best link
#https://confluence.slac.stanford.edu/display/PSDM/How+to+access+HDF5+data+from+Python

In [ ]:
# dim reduction ACP

In [ ]:
# Clasification - SVM

In [ ]:
# performance - SVM

In [ ]:
# Clasification - decision tree

In [ ]:
# performance - decision tree

In [ ]:
# Clasification - random forest

In [ ]:
# performance - random forest

## 4.Transferencia de aprendizaje: Fine tuning

In [ ]:
# feed Inception

In [ ]:
# performance

## 5.Resumen. Discusión resultados

#### Descriptores profundos

#### Fine tuning